## EDA & Data Cleaning

### Data description:

##### **train.csv**

- **customer_id**: código de identificación de cada cliente de Flow (puede tener asociados uno o más **account_id**)
- **account_id**: código de identificación de cada perfil de Flow (se corresponde con un único **customer_id**)
- **device_type**: indica el tipo de dispositivo desde el que se efectuó la visualización. Las categorías posibles son:
    - CLOUD: cliente web
    - PHONE: teléfono celular
    - STATIONARY: *smart* TV
    - STB: *set-top box*, el decodificador Flow
    - TABLET
- **asset_id**: código de identificación de cada activo (video) disponible en la plataforma
- **tunein**: fecha y hora de inicio de cada visualización
- **tuneout**: fecha y hora de finalización de cada visualización
- **resume**: variable *dummy* que indica si se reanuda un consumo anterior del mismo **asset_id**

##### **metadata.csv**
- **asset_id**: código de identificación de cada activo (video) disponible en Flow
- **content_id**: código de identificación que agrupa los distintos **asset_id** asociados a un mismo contenido (por ejemplo, cada episodio de una misma serie tiene su propio **asset_id**, mientras que la serie se identifica con un **content_id** único)
- **title**: título
- **reduced_title**: título reducido
- **episode_title**: título del episodio (válido para contenidos episódicos, como las series)
- **show_type**: tipo de show - las categorías son autorreferenciales con excepción de “Rolling”, que indica que se trata de una serie incompleta, y “Web”, la cual remite a  contenidos pensados íntegramente en formato digital (series web) -
- **released_year**: año de lanzamiento
- **country_of_origin**: país de origen – expresado con el código de dos letras propio del estándar internacional de normalización ISO 3166 -
- **category**: categoría o género al que pertenece el contenido - puede haber una o más -
- **keywords**: palabras clave o *tags* asociadas al contenido - puede haber una o más -
- **description**: descripción (sinopsis)
- **reduced_desc**: descripción (sinopsis) reducida
- **cast_first_name**: nombre y apellido de los actores y actrices principales
- **credits_first_name**: nombre y apellido del director o directora
- **run_time_min**: duración total, expresada en minutos
- **audience**: audiencia *target*
- **made_for_tv**: variable *dummy* que indica si el contenido fue hecho para TV
- **close_caption**: variable *dummy* que indica si el contenido posee subtítulos
- **sex_rating**: variable dummy *que* indica si el contenido tiene escenas de sexo explícitas
- **violence_rating**: variable *dummy* que indica si el contenido tiene escenas de violencia explícitas
- **language_rating**: variable *dummy* que indica si el contenido posee lenguaje que puede ser considerado ofensivo o inapropiado
- **dialog_rating**: variable *dummy* que indica si el contenido posee diálogos que pueden ser considerado ofensivos o inapropiados
- **fv_rating**: variable *dummy* que indica si el contenido tiene rating de FV, que corresponde a público infantil con violencia ficticia
- **pay_per_view**: variable *dummy* que indica si se trata de un alquiler
- **pack_premium_1**: variable *dummy* que indica si se trata de un contenido exclusivo del pack premium 1
- **pack_premium_2**: variable *dummy* que indica si se trata de un contenido exclusivo del pack premium 2
- **create_date**: fecha de creación del activo
- **modify_date**: fecha de modificación del activo
- **start_vod_date**: fecha desde la cual el activo se encuentra disponible en la plataforma
- **end_vod_date**: fecha de finalización de la disponibilidad del activo  en la plataforma

### Import Libraries

In [1]:
import pickle
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=2)

# pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

plt.rcParams['figure.figsize'] = [18, 6]

### Read data

In [2]:
df = pd.read_csv('data/train.csv', parse_dates=['tunein', 'tuneout'])
print(f'df shape: {df.shape}')
df.tail()

df shape: (3657801, 7)


,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
3657796,112339,3386,STB,330.0,2021-03-31 22:10:00,2021-03-31 22:22:00,0
3657797,112339,3386,STB,9931.0,2021-03-31 22:22:00,2021-03-31 22:34:00,1
3657798,112339,3386,STB,29929.0,2021-03-31 22:34:00,2021-03-31 22:46:00,0
3657799,112339,3386,STB,29929.0,2021-03-31 23:09:00,2021-03-31 23:13:00,1
3657800,112339,3386,STB,29929.0,2021-03-31 23:13:00,2021-03-31 23:24:00,0


In [3]:
df_meta = pd.read_csv('data/metadata.csv', sep=';',
                      parse_dates=['released_year','create_date','modify_date','start_vod_date','end_vod_date'])
print(f'df shape: {df_meta.shape}')
df_meta.tail()

df shape: (33144, 30)


,asset_id,content_id,title,reduced_title,episode_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
33139,3683,1979.0,T:1 Ep:03 Allen V. Farrow,Allen_V_Far_T1_E03,Episodio 3,Serie,2021.0,US,Drama/Documental,"Abusos,Familia,Juicio,Hechos Reales,Miniserie",Mientras Mia intenta proteger a Dylan del trau...,El equipo legal y de relaciones públicas de Al...,"Mia Farrow, Woody Allen, Dylan Farrow, Ronan F...",Dick Kirby,64.0,General,N,N,N,N,N,N,N,N,N,Y,2021-03-28 23:39:28+00:00,2021-03-29 00:06:09+00:00,2021-03-07 00:00:00+00:00,2021-11-20 23:59:00+00:00
33140,16810,1979.0,T:1 Ep:01 Allen V. Farrow,Allen_V_Far_T1_E01,Episodio 1,Serie,2021.0,US,Drama/Documental,"Abusos,Familia,Juicio,Hechos Reales,Miniserie",Mia Farrow habla sobre su relación con Woody A...,Mia Farrow habla sobre su relación con Woody A...,"Mia Farrow, Woody Allen, Dylan Farrow, Ronan F...",Dick Kirby,56.0,General,N,N,N,N,N,N,N,N,N,Y,2021-03-28 23:37:17+00:00,2021-03-29 00:06:01+00:00,2021-02-21 00:00:00+00:00,2021-11-20 23:59:00+00:00
33141,20468,943.0,T:2 Ep:34 Programa De Talentos,Prog_De_Tal_T2_E34,El antagonista,TV,2011.0,US,Infantil/Comedia,"Instituto,Amigos,Música",Fletcher crea una animación en la que parodia ...,Tres amigos acaban de convertirse en los nuevo...,Gülru arruinó el nuevo diseño de Gulfem.,Dan Signer,24.0,Teens,N,N,N,N,N,N,N,N,N,N,2021-02-23 10:57:32+00:00,2021-02-24 20:22:16+00:00,2021-02-17 00:00:00+00:00,2021-03-19 23:59:59+00:00
33142,3779,1957.0,T:2 Ep:05 Batwoman,Batwoman_T2_E05,Gore sobre lienzo,Serie,2021.0,US,Acción/Aventura,"DC Comics,Superhéroes,Mujeres,Comics",La tensión aumenta en el equipo cuando Luke y ...,La tensión aumenta en el equipo cuando Luke y ...,"Javicia Leslie, Rachel Skarsten, Meagan Tandy,...",Holly Dale,42.0,General,N,N,N,N,N,N,N,N,N,Y,2021-03-26 09:49:52+00:00,2021-03-26 10:06:04+00:00,2021-02-26 00:00:00+00:00,2024-01-28 23:59:00+00:00
33143,30238,1041.0,T:1 Ep:25 Manzana y Cebollín,Manzana_y_Ce_T1_E24,El corto de Manzana,TV,2017.0,US,Infantil/Dibujos Animados,"Amigos,Travesuras",Manzana tiene que ser alto para poder enamorarse.,Manzana tiene que ser alto para poder enamorarse.,"Sayed Badreya, Richard Ayoade, George Gendi",Benton Connor,11.0,Teens,N,N,N,N,N,N,N,N,N,N,2021-02-05 17:40:30+00:00,2021-02-05 21:06:06+00:00,2021-02-01 00:00:00+00:00,2021-02-28 23:59:00+00:00


### Nulls - df_train

In [4]:
df.isnull().sum()

customer_id     0
account_id      0
device_type    29
asset_id       22
tunein          0
tuneout         0
resume          0
dtype: int64

#### device_type

In [5]:
df[df.device_type.isnull()].iloc[:5]

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
528885,18693,105629,NaN,10939.0,2021-01-17 16:04:00,2021-01-17 17:07:00,0
666823,23439,6012,NaN,6371.0,2021-01-17 21:55:00,2021-01-17 21:58:00,0
737730,25729,46974,NaN,3481.0,2021-01-13 11:30:00,2021-01-13 11:33:00,0
737731,25729,46974,NaN,3481.0,2021-01-13 11:33:00,2021-01-13 11:36:00,0
737732,25729,46974,NaN,3481.0,2021-01-13 11:36:00,2021-01-13 12:36:00,0


In [6]:
# Vemos si hay usuarios con solo esos registros nulos

users_with_null_devices = df[df.account_id.isin(df[df.device_type.isnull()].account_id)].groupby('account_id')['device_type'].agg(lambda x: x.isnull().all())
users_with_null_devices

account_id
6012      False
6983      False
14892     False
17621     False
25402     False
46974     False
52544     False
71627     False
74384     False
82302     False
100979    False
101874    False
102111    False
105629    False
108985    False
110083     True
Name: device_type, dtype: bool

El usuario **110083** solo aparece sin *device_type*.   

In [7]:
# Vamos a completar ese usuario (110083) con el *device_type* mas comun (moda)
users_without_any_device = users_with_null_devices[users_with_null_devices == True].index.values
df.loc[df.account_id.isin(users_without_any_device),
       'device_type'] = df.device_type.mode().values[0]

# Vamos los demas nulls con la moda el *device_type* moda de cada user
users_with_device = users_with_null_devices[users_with_null_devices == False].index.values
device_by_user = df[(df.account_id.isin(users_with_device))].groupby('account_id')['device_type'].agg(lambda x: x.mode()[0])

for user_id, device_type in device_by_user.items():
    df.loc[(df.account_id == user_id) & (df.device_type.isnull()), 'device_type'] = device_type

#### asset_id

In [8]:
df[df.asset_id.isnull()].iloc[:5]

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
106890,3746,28992,STB,NaN,2021-03-20 12:50:00,2021-03-20 12:52:00,1
120175,4277,29416,STB,NaN,2021-03-16 00:29:00,2021-03-16 00:36:00,0
146642,5174,30114,STB,NaN,2021-03-28 22:35:00,2021-03-28 22:39:00,0
219038,7685,32112,STB,NaN,2021-03-20 21:15:00,2021-03-20 21:55:00,0
249196,8809,33030,PHONE,NaN,2021-03-20 15:16:00,2021-03-20 15:18:00,0


In [9]:
# Vemos si hay usuarios con solo esos registros nulos

users_with_null_asset = df[df.account_id.isin(df[df.asset_id.isnull()].account_id)].groupby('account_id')['asset_id'].agg(lambda x: x.isnull().all())
users_with_null_asset

account_id
10784     False
13058     False
21345     False
23657     False
24742     False
28992     False
29416      True
30114     False
32112     False
33030     False
39757     False
46629     False
50616     False
56086     False
60401     False
62625     False
71182     False
76218     False
76429     False
86969     False
91253      True
108646    False
Name: asset_id, dtype: bool

In [10]:
# Vamos a completar los usuarios (29416, 91253) con el *asset_id* mas comun (moda)

users_without_any_asset = users_with_null_asset[users_with_null_asset == True].index.values
df.loc[df.account_id.isin(users_without_any_asset) & (df.asset_id.isnull()), 'asset_id'] = df.asset_id.mode()[0]

# Vamos a completar los demas usuarios con la moda de cada uno
users_with_asset = users_with_null_asset[users_with_null_asset == False].index.values
asset_by_user = df[(df.account_id.isin(users_with_asset))].groupby('account_id')['asset_id'].agg(lambda x: x.mode()[0])

for user_id, asset_id in asset_by_user.items():
    df.loc[(df.account_id == user_id) & (df.asset_id.isnull()), 'asset_id'] = asset_id

In [11]:
# chequeamos que no queden nulls

df.isnull().sum().any()

False

### Nulls - df_meta

In [12]:
df_meta.isnull().sum().sort_values(ascending=False)

credits_first_name    12554
cast_first_name        8732
episode_title          4147
content_id               21
show_type                 4
country_of_origin         4
keywords                  2
description               2
audience                  1
end_vod_date              0
title                     0
reduced_title             0
released_year             0
category                  0
reduced_desc              0
run_time_min              0
start_vod_date            0
made_for_tv               0
close_caption             0
sex_rating                0
violence_rating           0
language_rating           0
dialog_rating             0
fv_rating                 0
pay_per_view              0
pack_premium_1            0
pack_premium_2            0
create_date               0
modify_date               0
asset_id                  0
dtype: int64

#### credits_first_name

In [13]:
df_meta.credits_first_name.iloc[:5]

0           Daniel Barone
1    Juan Carlos Maneglia
2        Marko Mäkilaakso
3         Sebastián Lelio
4             J.J. Abrams
Name: credits_first_name, dtype: object

In [14]:
# Para simplificar vamos a llenar con "unknow-director" los nulls

df_meta.credits_first_name.fillna("unknow-director", inplace=True)

#### cast_first_name

In [15]:
df_meta.cast_first_name.iloc[:5]

0    Rodrigo De la Serna, Paola Krum, Fernán Mirás,...
1             Celso Franco, Víctor Sosa, Lali Gonzalez
2    Mark Arnold, Harry Lister Smith, Vanessa Grass...
3    Daniela Vega, Francisco Reyes, Luis Gnecco, Am...
4    Chris Pine, Zachary Quinto, Leonard Nimoy, Eri...
Name: cast_first_name, dtype: object

In [16]:
# veamos si podemos copiar el director desde otros contenidos

# chequemos si todos los registros de ese contenido tienen valores nullos
df_meta[df_meta.cast_first_name.isnull()].groupby('content_id')['cast_first_name'].agg(lambda x: x.isnull().all()).all()

True

In [17]:
# Para simplificar vamos a llenar con "unknow-actors" los nulls

df_meta.cast_first_name.fillna("unknow-actors", inplace=True)

#### episode_title

In [18]:
df_meta[['reduced_title', 'episode_title']].iloc[:5]

,reduced_title,episode_title
0,Tiempos_Compul_E17,Episodio 17
1,7_Cajas,NaN
2,La_Maldicion_de_las,NaN
3,Una_Mujer_Fantastic,NaN
4,Star_Trek,NaN


- Esta variable no parece relevante, ya que nos piden recomendar contenido y no episodios de una misma serie. Se borra

In [19]:
df_meta.drop(columns=['episode_title'], inplace=True)

#### content_id

In [20]:
df_meta[df_meta.content_id.isnull()].show_type.unique()

array(['Tutorial'], dtype=object)

- Vemos que todos los `content_id` nulos son de tutoriales de flow. Vamos a borrar esos datos pero guardamos los `asset_id`

In [21]:
assets_dropped = df_meta[df_meta.content_id.isnull()].asset_id.unique()

In [22]:

df_meta.drop(df_meta[df_meta.content_id.isnull()].index, inplace=True)

#### show_type

In [23]:
df_meta.show_type.value_counts()

TV              15681
Serie           10905
Película         4025
Web              1386
Gaming            844
Rolling           275
Series,Serie        3
Name: show_type, dtype: int64

In [24]:
df_meta[df_meta.show_type.isnull()]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
3187,14004,9.0,Star Trek: En la oscuridad,Star_Trek__En_la_os,NaN,2013.0,US,Acción/Ciencia Ficción,"Aventuras,Saga,Futurismo,Galaxia,Criaturas,Ali...",La tripulación de la nave Enterprise recibe la...,La tripulación de la Enterprise recibe la orde...,"Chris Pine, Zachary Quinto, Zoe Saldana, Bened...",J.J. Abrams,132.0,General,N,N,N,N,N,N,N,N,N,N,2020-11-16 21:16:01+00:00,2020-12-09 17:10:20+00:00,2020-11-20 00:00:00+00:00,2020-12-14 23:59:59+00:00
6421,11154,948.0,Un verano en pantalones,Un_Verano_En_Pantal,NaN,2005.0,US,Comedia,"Mujeres,De Libros,Amigos,Viajes",Cuatro mejores amigas traman un plan para mant...,Cuatro mejores amigas traman un plan para mant...,"Amber Tamblyn, Alexis Bledel, America Ferrera,...",Ken Kwapis,131.0,Mujeres,N,N,N,N,N,N,N,N,N,N,2020-12-29 18:10:46+00:00,2020-12-29 20:06:05+00:00,2020-12-02 00:00:00+00:00,2021-01-01 23:59:00+00:00
7674,27728,9.0,Star Trek: En la oscuridad,Star_Trek_En_la_DOB,NaN,2013.0,US,Acción/Ciencia Ficción,"Aventuras,Saga,Futurismo,Galaxia,Criaturas,Ali...",La tripulación de la nave Enterprise recibe la...,La tripulación de la Enterprise recibe la orde...,"Chris Pine, Zachary Quinto, Zoe Saldana, Bened...",J.J. Abrams,132.0,General,N,N,N,N,N,N,N,N,N,N,2020-12-11 15:09:21+00:00,2020-12-24 15:06:09+00:00,2020-12-15 00:00:00+00:00,2022-11-20 23:59:00+00:00
32927,24509,4315.0,El día que el deporte se detuvo,El_dia_que_el_depor,NaN,2021.0,US,Documental/Deporte,"Pandemias,Deportistas,De Color,Sociedad","Documental original de HBO. Antoine Fuqua, rel...",Documental original de HBO que cuenta la histo...,unknow-actors,Antoine Fuqua,85.0,General,N,N,N,N,N,N,N,N,N,Y,2021-03-29 16:16:36+00:00,2021-03-29 17:06:04+00:00,2021-03-24 00:00:00+00:00,2022-01-10 23:59:00+00:00


- Vamos a completar de 1 dado que son pocos registros nulos

In [25]:
# "Star Trek: En la oscuridad" es una "pelicula"

df_meta.loc[(df_meta.show_type.isnull()) & (df_meta.content_id == 9), 'show_type'] = 'Película'

# "Un verano en pantalones" es una "pelicula"
df_meta.loc[(df_meta.show_type.isnull()) & (df_meta.content_id == 948), 'show_type'] = 'Película'

# "El día que el deporte se detuvo" es una "Documental" completamos con "pelicula" tambien
df_meta.loc[(df_meta.show_type.isnull()) & (df_meta.content_id == 4315.0), 'show_type'] = 'Película'

#### country_of_origin

In [26]:
df_meta[df_meta.country_of_origin.isnull()]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
7287,31314,1124.0,T:6 Ep:06 Dr. Pol,Dr_Pol_T6_E06,TV,2016.0,NaN,Reality/Animales,"Veterinarios,Animales,Enfermedades","Cuellos mordidos, balazos y estruendosos gruñi...","Cuellos mordidos, balazos y estruendosos gruñi...","Jan Pol, Charles Pol, Diane Pol",Ted Duvall,43.0,General,N,N,N,N,N,N,N,N,N,N,2018-05-10 02:55:24+00:00,2019-03-26 12:40:02+00:00,2018-05-03 00:00:00+00:00,2021-01-16 23:59:59+00:00
7320,22004,1124.0,T:7 Ep:16 Dr. Pol,Dr_Pol_T7_E16,TV,2017.0,NaN,Reality/Animales,"Veterinarios,Animales,Enfermedades","Esta semana en 'El increíble Dr. Pol', la doct...","Esta semana en 'El increíble Dr. Pol', la doct...","Jan Pol, Charles Pol, Diane Pol",Ted Duvall,42.0,General,N,N,N,N,N,N,N,N,N,N,2019-04-03 21:34:38+00:00,2019-10-17 11:26:23+00:00,2018-12-19 00:00:00+00:00,2021-01-16 23:59:00+00:00
7321,9452,1124.0,T:7 Ep:17 Dr. Pol,Dr_Pol_T7_E17,TV,2017.0,NaN,Reality/Animales,"Veterinarios,Animales,Enfermedades",Es la primera semana de trabajo de la doctora ...,Es la primera semana de trabajo de la doctora ...,"Jan Pol, Charles Pol, Diane Pol",Ted Duvall,43.0,General,N,N,N,N,N,N,N,N,N,N,2019-04-03 21:37:39+00:00,2019-10-17 11:26:24+00:00,2018-12-26 00:00:00+00:00,2021-01-16 23:59:00+00:00
17406,8946,2264.0,My Mother's Future Husband,My_Mothers_Future,Película,2014.0,NaN,Romance/Comedia,"Melodrama,Tragicómica,Familia,Parejas",Cuando Headly experimenta su primer enamoramie...,Cuando Headly experimenta su primer enamoramie...,"Frank Cassini, Burkely Duffield, Matreya Fedor",George Erschbamer,89.0,Mujeres,N,N,N,N,N,N,N,N,Y,N,2019-04-04 16:32:11+00:00,2019-10-17 11:26:26+00:00,2019-04-03 00:00:00+00:00,2022-03-31 23:59:00+00:00


- Como son los completamos a mano, una serie es de US y la otra de CA (Canada)

In [27]:
# completamos con 'US'

df_meta.loc[(df_meta.country_of_origin.isnull()) & (df_meta.content_id == 1124), 'country_of_origin'] = 'US'

# completamos con 'CA'

df_meta.loc[(df_meta.country_of_origin.isnull()) & (df_meta.content_id == 2264), 'country_of_origin'] = 'CA'

#### keywords

In [28]:
df_meta[df_meta.keywords.isnull()]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
29761,25451,3914.0,La supremacía Bourne,La_supremacia_Bourn,Película,2004.0,US,Acción/Suspenso,NaN,Cuando el viceprimer ministro chino es brutalm...,"Cuando un ministro chino es asesinado, la CIA ...","Matt Damon, Franka Potente, Brian Cox, Karl Ur...",Paul Greengrass,102.0,Hombres,N,N,N,N,N,N,N,N,Y,N,2021-02-09 12:40:00+00:00,2021-02-09 13:06:09+00:00,2021-02-07 00:00:00+00:00,2021-06-30 23:59:00+00:00
30564,27065,1175.0,T:2 Ep:75 Drama total: La guardería,Drama_total__T2_E75,TV,2019.0,US,Infantil/Dibujos Animados,NaN,Tus personajes favoritos de Isla del Drama reg...,Tus personajes favoritos de Isla del Drama reg...,unknow-actors,unknow-director,10.0,Teens,N,N,N,N,N,N,N,N,N,N,2021-03-01 23:10:13+00:00,2021-03-02 05:06:05+00:00,2021-02-20 00:00:00+00:00,2021-04-06 23:59:00+00:00


- Compleamos con valores a mano (para *Drama total* tomamos el valor de otros registros)

In [29]:
df_meta.loc[(df_meta.content_id == 1175), 'keywords']

7635          Travesuras
7971          Travesuras
8763          Travesuras
8781          Travesuras
9152          Travesuras
9205          Travesuras
10077         Travesuras
11403         Travesuras
11709         Travesuras
12381         Travesuras
12382         Travesuras
12697         Travesuras
30148    Niños,Ingenioso
30564                NaN
30826    Niños,Ingenioso
31765    Niños,Ingenioso
Name: keywords, dtype: object

In [30]:
# Completamos con "Travesuras"

df_meta.loc[(df_meta.keywords.isnull()) & (df_meta.content_id == 1175), 'keywords'] = \
    df_meta[(~df_meta.keywords.isnull()) & (df_meta.content_id == 1175)].keywords.mode()[0]

# Completamos con ""
df_meta.loc[(df_meta.keywords.isnull()) & (df_meta.content_id == 3914), 'keywords'] = 'Espionaje,Adrenalina,Terroristas,Agentes'

#### description

In [31]:
df_meta[df_meta.description.isnull()]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
16238,5514,2168.0,T:1 Ep:03 Camping,Camping_T1_E03,Serie,2018.0,US,Comedia,"Vacaciones,Parejas,Amigos,Optimistas,Fiestas,V...",NaN,Walt se sincera en un viaje de pesca. Jandice ...,"Juliette Lewis, Jennifer Garner, Ione Skye, Da...",Jenni Konner,28.0,General,N,N,N,N,N,N,N,N,N,Y,2018-11-20 16:33:34+00:00,2020-09-30 12:08:37+00:00,2018-11-19 00:00:00+00:00,2021-03-31 23:59:59+00:00
16330,21755,2168.0,T:1 Ep:04 Camping,Camping_T1_E04,Serie,2018.0,US,Comedia,"Vacaciones,Parejas,Amigos,Optimistas,Fiestas,V...",NaN,Un cambio de última hora lleva a Kathryn a col...,"Juliette Lewis, Jennifer Garner, Ione Skye, Da...",Jenni Konner,27.0,General,N,N,N,N,N,N,N,N,N,Y,2018-11-26 15:49:48+00:00,2020-09-30 12:11:40+00:00,2019-11-26 00:00:00+00:00,2021-03-31 23:59:59+00:00


- Completamos con la "*reduced_desc*"

In [32]:
df_meta.loc[df_meta.description.isnull(), 'description'] = df_meta[df_meta.description.isnull()].reduced_desc.values

#### audience

In [33]:
df.tail()

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
3657796,112339,3386,STB,330.0,2021-03-31 22:10:00,2021-03-31 22:22:00,0
3657797,112339,3386,STB,9931.0,2021-03-31 22:22:00,2021-03-31 22:34:00,1
3657798,112339,3386,STB,29929.0,2021-03-31 22:34:00,2021-03-31 22:46:00,0
3657799,112339,3386,STB,29929.0,2021-03-31 23:09:00,2021-03-31 23:13:00,1
3657800,112339,3386,STB,29929.0,2021-03-31 23:13:00,2021-03-31 23:24:00,0


In [34]:
df_meta.audience.value_counts()

General       13637
Mujeres        4553
Juvenil        4453
Teens          3840
Preescolar     2301
Niños          2292
Hombres        1568
Familiar        438
NIños            36
Gaming            4
Name: audience, dtype: int64

In [35]:
df_meta[df_meta.audience.isnull()]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
14523,26900,2064.0,T:1 Ep:08 Refugios Extremos,Refugios_Ext_T1_E08,TV,2014.0,US,Reality/Restauración,"Naturaleza,Decoración,Diseño",John y su esposa son italianos y desean constr...,John y su esposa son italianos y desean constr...,unknow-actors,George Verschoor,43.0,NaN,N,N,N,N,N,N,N,N,N,N,2018-01-17 17:51:55+00:00,2019-06-12 03:04:45+00:00,2018-01-05 00:00:00+00:00,2029-12-31 23:59:00+00:00


In [36]:
df_meta.loc[(df_meta.content_id == 2064)]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
14523,26900,2064.0,T:1 Ep:08 Refugios Extremos,Refugios_Ext_T1_E08,TV,2014.0,US,Reality/Restauración,"Naturaleza,Decoración,Diseño",John y su esposa son italianos y desean constr...,John y su esposa son italianos y desean constr...,unknow-actors,George Verschoor,43.0,NaN,N,N,N,N,N,N,N,N,N,N,2018-01-17 17:51:55+00:00,2019-06-12 03:04:45+00:00,2018-01-05 00:00:00+00:00,2029-12-31 23:59:00+00:00


In [37]:
audience_mode = df_meta.audience.value_counts().index.values[0]

df_meta.loc[(df_meta.audience.isnull()), 'audience'] = audience_mode

In [38]:
# chequeamos que no queden nulls

df_meta.isnull().sum().any()

False

#### run_time_min

- Vamos a unificar los registros con run_time_min == 0

In [39]:
df_meta[df_meta.run_time_min == 0].shape

(114, 29)

In [40]:
# Vamos a unificar esto usando la moda para cada content_id

data_by_content = df_meta.groupby('content_id')['run_time_min'].agg(lambda x: x.max())
data_by_content.name = 'run_time_min'

df_meta = pd.merge(df_meta.drop(columns=['run_time_min']), data_by_content, how='inner', left_on='content_id', right_index=True)

In [41]:
# Completamos with mode

df_meta.loc[df_meta.run_time_min == 0, 'run_time_min'] = int(df_meta.run_time_min.mean())

### Check Consistency
Chequeamos que el dataset tiene consistencia entre sus registros

- Chequeamos los `assets_id` borrados (los tutoriales)

In [42]:
df[df.asset_id.isin(assets_dropped)].iloc[:5]

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
27952,945,26773,STB,3938.0,2021-03-20 17:44:00,2021-03-20 17:46:00,0
27953,945,26773,STB,28382.0,2021-03-20 17:50:00,2021-03-20 17:53:00,0
69652,2313,27844,STB,3938.0,2021-03-09 00:11:00,2021-03-09 00:13:00,0
95034,3297,28604,STATIONARY,3938.0,2021-02-26 00:45:00,2021-02-26 00:47:00,0
135586,4830,29840,STB,3938.0,2021-03-27 22:24:00,2021-03-27 22:26:00,0


Vamos a cambiarlos por el contenido menos visto

In [43]:
asset_mode = df.asset_id.value_counts().index[-1]
df.loc[df.asset_id.isin(assets_dropped), 'asset_id'] = asset_mode

- Vamos a ver si algun ***account_id*** corresponde a mas de un ***customer_id***

In [44]:
customers_by_account = df.groupby('account_id')['customer_id'].agg(lambda x: len(set(x)))
df[df.account_id.isin( customers_by_account[customers_by_account > 1].index )]

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume


-  Veamos si algun ***asset_id*** corresponde a mas de un ***content_id***

In [45]:
contents_by_asset = df_meta.groupby('asset_id')['content_id'].agg(lambda x: len(set(x)))
df_meta[df_meta.asset_id.isin( contents_by_asset[contents_by_asset > 1].index )]

,asset_id,content_id,title,reduced_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date,run_time_min


-  Veamos si algun ***content_id*** corresponde a mas de un ***show_type***

In [46]:
data_by_content = df_meta.groupby('content_id')['show_type'].agg(lambda x: set(x))
data_by_content[data_by_content.map(len) > 1]

content_id
185.0               {TV, Serie}
691.0     {Series,Serie, Serie}
778.0               {TV, Serie}
1209.0        {Película, Serie}
1281.0              {TV, Serie}
1380.0    {Series,Serie, Serie}
1422.0              {TV, Serie}
2135.0         {Rolling, Serie}
2994.0              {TV, Serie}
3097.0              {TV, Serie}
3285.0             {Web, Serie}
3413.0        {Película, Serie}
3704.0              {TV, Serie}
3739.0              {TV, Serie}
3836.0              {TV, Serie}
3840.0              {TV, Serie}
Name: show_type, dtype: object

In [47]:
# Vamos a unificar esto usando la moda para cada content_id

data_by_content = df_meta.groupby('content_id')['show_type'].agg(lambda x: x.mode()[0])
data_by_content.name = 'show_type'

df_meta = pd.merge(df_meta.drop(columns=['show_type']), data_by_content, how='inner', left_on='content_id', right_index=True)

-  Veamos si algun ***content_id*** corresponde a mas de un ***country_of_origin***

In [48]:
data_by_content = df_meta.groupby('content_id')['country_of_origin'].agg(lambda x: set(x))
data_by_content[data_by_content.map(len) > 1].iloc[:5]

content_id
59.0          {US, AR}
71.0          {FM, US}
476.0         {AU, US}
633.0    {US, Francia}
654.0         {GB, US}
Name: country_of_origin, dtype: object

In [49]:
# Vamos a unificar esto usando la moda para cada content_id

data_by_content = df_meta.groupby('content_id')['country_of_origin'].agg(lambda x: x.mode()[0])
data_by_content.name = 'country_of_origin'

df_meta = pd.merge(df_meta.drop(columns=['country_of_origin']), data_by_content, how='inner', left_on='content_id', right_index=True)

### Categories

In [50]:
# Clean up the category strings 

df_meta.category = df_meta.category.str.lower()\
                                   .replace('á', 'a', regex=True)\
                                   .replace('é', 'e', regex=True)\
                                   .replace('í', 'i', regex=True)\
                                   .replace('ó', 'o', regex=True)\
                                   .replace('ú', 'u', regex=True)\
                                   .replace(', ', '/', regex=True)\
                                   .replace('familiar', 'familia', regex=True)\
                                   .replace('aventuras', 'aventura', regex=True)\
                                   .replace('musical', 'musica', regex=True)

# Binarize columns

df_cat = pd.get_dummies(df_meta.category.str.split("/", expand=True)
                       ).groupby(lambda x: x.split('_')[-1], axis=1).sum()

categories_list = df_cat.columns.values.tolist()

# Join categories columns to df

df_meta = pd.concat([df_meta.drop(columns=['category']), df_cat], axis=1)

### Dummies variables

#### Binarizamos estas variables

In [51]:
dummies_cols = ['made_for_tv', 'close_caption', 'sex_rating', 'violence_rating', 'language_rating', 
                'dialog_rating', 'fv_rating', 'pay_per_view', 'pack_premium_1', 'pack_premium_2']

In [52]:
for col in dummies_cols:
    print(f'{col}:', df_meta[col].unique())

made_for_tv: ['N' 'Y']
close_caption: ['N']
sex_rating: ['N']
violence_rating: ['N']
language_rating: ['N']
dialog_rating: ['N']
fv_rating: ['N']
pay_per_view: ['N' 'Y']
pack_premium_1: ['N' 'Y']
pack_premium_2: ['N' 'Y']


In [53]:
for col in dummies_cols:
    df_meta[col].replace({'N': 0, 'Y': 1}, inplace=True)

- Drop variables with an unique value

In [54]:
df_meta.drop(columns=['close_caption', 'sex_rating', 'violence_rating', 'language_rating', 'dialog_rating', 'fv_rating'], inplace=True)

#### made_for_tv

In [55]:
for_tv_by_content = df_meta.groupby('content_id')['made_for_tv'].agg(set)
for_tv_by_content[for_tv_by_content.map(len) > 1]

content_id
74.0      {0, 1}
698.0     {0, 1}
1008.0    {0, 1}
1009.0    {0, 1}
1029.0    {0, 1}
1044.0    {0, 1}
1046.0    {0, 1}
1047.0    {0, 1}
1155.0    {0, 1}
1160.0    {0, 1}
1374.0    {0, 1}
1375.0    {0, 1}
1415.0    {0, 1}
1582.0    {0, 1}
1621.0    {0, 1}
1792.0    {0, 1}
1908.0    {0, 1}
2028.0    {0, 1}
2055.0    {0, 1}
2056.0    {0, 1}
2061.0    {0, 1}
2072.0    {0, 1}
2082.0    {0, 1}
2087.0    {0, 1}
2088.0    {0, 1}
2171.0    {0, 1}
2205.0    {0, 1}
2542.0    {0, 1}
Name: made_for_tv, dtype: object

#### pay_per_view

In [56]:
pay_by_content = df_meta.groupby('content_id')['pay_per_view'].agg(set)
pay_by_content[pay_by_content.map(len) > 1]

content_id
1.0       {0, 1}
9.0       {0, 1}
10.0      {0, 1}
70.0      {0, 1}
71.0      {0, 1}
           ...  
3553.0    {0, 1}
3554.0    {0, 1}
3702.0    {0, 1}
4082.0    {0, 1}
4139.0    {0, 1}
Name: pay_per_view, Length: 138, dtype: object

#### pack_premium_1

In [57]:
premium_1_by_content = df_meta.groupby('content_id')['pack_premium_1'].agg(set)
premium_1_by_content[premium_1_by_content.map(len) > 1]

content_id
3.0       {0, 1}
9.0       {0, 1}
10.0      {0, 1}
23.0      {0, 1}
24.0      {0, 1}
           ...  
3141.0    {0, 1}
3487.0    {0, 1}
3553.0    {0, 1}
3625.0    {0, 1}
3702.0    {0, 1}
Name: pack_premium_1, Length: 75, dtype: object

#### pack_premium_2

In [58]:
premium_2_by_content = df_meta.groupby('content_id')['pack_premium_2'].agg(set)
premium_2_by_content[premium_2_by_content.map(len) > 1].iloc[:7]

content_id
87.0     {0, 1}
126.0    {0, 1}
136.0    {0, 1}
173.0    {0, 1}
188.0    {0, 1}
190.0    {0, 1}
643.0    {0, 1}
Name: pack_premium_2, dtype: object

- Estas variables tienen diferentes valores para el mismo `content_id`, pero para saber que valor usar primero deberiamos unir con el **df_tran** para saber cuales `asset_id` se usan

### Strings columns

#### Drop redundant columns

In [59]:
df_meta.drop(columns=['reduced_title', 'reduced_desc'], inplace=True)

#### Title
Limpiamos la info de Episodio y Temporada del titulo

In [60]:
df_meta.title = df_meta.title.str.lower()\
                             .replace('t:\d{1,4} ?', '', regex=True)\
                             .replace('ep:\d{1,4} ?', '', regex=True)\
                             .replace('á', 'a', regex=True)\
                             .replace('é', 'e', regex=True)\
                             .replace('í', 'i', regex=True)\
                             .replace('ó', 'o', regex=True)\
                             .replace('ú', 'u', regex=True)\
                             .replace('(null|n/a)', '', regex=True)

In [61]:
# Vamos a unificar esto usando la moda para cada content_id

data_by_content = df_meta.groupby('content_id')['title'].agg(lambda x: x[x != ''].mode()[0])
data_by_content.name = 'title'

df_meta = pd.merge(df_meta.drop(columns=['title']), data_by_content, how='inner', left_on='content_id', right_index=True)

#### keywords

In [62]:
df_meta.keywords = df_meta.keywords.str.lower()\
                           .replace('á', 'a', regex=True)\
                           .replace('é', 'e', regex=True)\
                           .replace('í', 'i', regex=True)\
                           .replace('ó', 'o', regex=True)\
                           .replace('ú', 'u', regex=True)\
                           .replace(', ', ',', regex=True)\
                           .replace('t,i,t,l,e', '', regex=True)\
                           .replace('(null|n/a)(?=[,$])', '', regex=True)

In [63]:
# Vamos a unificar esto usando la moda para cada content_id

data_by_content = df_meta.groupby('content_id')['keywords'].agg(lambda x: x[x != ''].mode()[0])
data_by_content.name = 'keywords'

df_meta = pd.merge(df_meta.drop(columns=['keywords']), data_by_content, how='inner', left_on='content_id', right_index=True)

### Save Results

In [64]:
df.to_csv('data/train_cleaned.csv', index=False)

In [65]:
df_meta.to_csv('data/metadata_cleaned.csv', index=False)

----
---
---
---

### Datetime columns

In [3]:
df = pd.read_csv('data/train_cleaned.csv', parse_dates=['tunein', 'tuneout'])

df.tail(3)

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
3657798,112339,3386,STB,29929.0,2021-03-31 22:34:00,2021-03-31 22:46:00,0
3657799,112339,3386,STB,29929.0,2021-03-31 23:09:00,2021-03-31 23:13:00,1
3657800,112339,3386,STB,29929.0,2021-03-31 23:13:00,2021-03-31 23:24:00,0


#### df_train
Vamos a unir los eventos que resumen un contenido ya visto

In [4]:
df.shape

(3657801, 7)

In [5]:
df.account_id.nunique(), df.asset_id.nunique()

(113881, 23978)

In [6]:
# Esta columna indica cuanto tiempo se vio un contendio

df.loc[:, 'min_watching'] = (df.tuneout - df.tunein).dt.seconds / 60

In [7]:
# Funcion para unir las filas que rsumen un contenido ya comenzado

agg_funcs = {
    'customer_id': 'first',
    'account_id': 'first',
    'asset_id': 'first',
    'device_type': 'first',
    'tunein': 'first',
    'tuneout': 'last',
    'resume': 'first',
    'min_watching': 'sum'
}

def joins_views(df_gr):
    df_gr = (
        df_gr.groupby(df_gr['resume'].eq(0).cumsum(), as_index=False)
        .agg(agg_funcs)
    )

    return df_gr

In [ ]:
%%time

# Creamos un df con solo los vistas que agrupados tengan mas de 1 evento
df_cumsum = df[df.duplicated(subset=['account_id', 'asset_id'], keep=False)].copy()

# Ordenamos los eventos tunein
df_cumsum.sort_values('tunein', inplace=True)

# Aplicamos la funcion para unir los grupos de eventos
df_cumsum = df_cumsum.groupby(['account_id', 'asset_id'], as_index=False).apply(joins_views)

# Borramos el index de 'account_id', 'asset_id'
df_cumsum = df_cumsum.reset_index(drop=True)

df_cumsum.shape

In [ ]:
df_cumsum.tail()

In [ ]:
df = pd.concat([df[~df.duplicated(subset=['account_id', 'asset_id'], keep=False)], df_cumsum])
df.shape

- Chequeamos consitencia ( Vemos si tenemos los mismos users y el mismo contenido que antes )

In [ ]:
df.asset_id.nunique(), df.account_id.nunique()

- Vamos a quedarnos solo con la hora que ven los programas los usuarios y borramos el `tuneout`

In [ ]:
df.drop(columns=['tuneout'], inplace=True)

#### df_meta

In [ ]:
df_meta = pd.read_csv('data/metadata_cleaned.csv',
                      parse_dates=['create_date', 'modify_date', 'start_vod_date', 'end_vod_date'])

df = pd.merge(df, df_meta, how='left', on='asset_id')
df.tail(3)

- Vamos a dejar como fecha de finalizacion solo en dia y borramos las demas

In [ ]:
df.loc[:, 'end_vod_date'] = df.end_vod_date.dt.date

df.drop(columns=['create_date', 'modify_date', 'start_vod_date'], inplace=True)

### Ranking time wathching based
Vamos a armar un ranking basado en el tiempo de que vio un contenido un usuario

---

In [153]:
df[
    df.ranking == 1
][['account_id', 'asset_id', 'content_id', 'title', 'run_time_min', 'min_watching', 'ranking']]

,account_id,asset_id,content_id,title,run_time_min,min_watching,ranking
4,3388,29743.0,3487.0,socias en guerra,84.0,16.0,1
6,3388,16202.0,2100.0,the resident,43.0,8.0,1
7,3388,16571.0,2100.0,the resident,43.0,5.0,1
9,3388,1869.0,2040.0,this is us,53.0,6.0,1
13,3389,28611.0,3225.0,los intrusos,90.0,4.0,1
...,...,...,...,...,...,...,...
3089004,113827,5467.0,2798.0,la reconstruccion,90.0,8.0,1
3089008,113828,4668.0,3232.0,la paciente,95.0,4.0,1
3089029,113859,20247.0,1917.0,la decision,103.0,2.0,1
3089031,113862,5168.0,3386.0,estafadoras de wall street,109.0,8.0,1


---

---

In [ ]:
# vamos a setear como maximo tiempo que alguien vio un contenido la duracion de este

df.loc[df.min_watching > df.run_time_min, 'min_watching'] = df.run_time_min

df.loc[:, 'ranking'] = (df.min_watching / df.run_time_min) * 10

df.loc[df.ranking < 1, 'ranking'] = 1

df.loc[:, 'ranking'] = df.ranking.astype(int)

df.tail()

In [ ]:
df.ranking.describe().astype(int)

### Save Results

In [ ]:
df.to_csv('data/data_cleaned_joined.csv', index=False)

---

In [18]:
whos

Variable           Type         Data/Info
-----------------------------------------
agg_funcs          dict         n=8
datetime           type         <class 'datetime.datetime'>
df                 DataFrame             customer_id  acc<...>089038 rows x 68 columns]
df_cumsum          DataFrame             customer_id  acc<...>1133220 rows x 8 columns]
df_meta            DataFrame           asset_id  content_<...>[33123 rows x 63 columns]
joins_views        function     <function joins_views at 0x7f86cf0fb0d0>
np                 module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd                 module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
pickle             module       <module 'pickle' from '/u<...>lib/python3.8/pickle.py'>
plt                module       <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
sns                module       <module 'seaborn' from '/<...>ges/seaborn/__init__.py'>
train_test_split   function     <function t

In [19]:
!head -n 1 data/data_cleaned_joined.csv

customer_id,account_id,device_type,asset_id,tunein,resume,min_watching,tunein_hour,content_id,released_year,description,cast_first_name,credits_first_name,audience,made_for_tv,pay_per_view,pack_premium_1,pack_premium_2,end_vod_date,run_time_min,show_type,country_of_origin,accion,animacion,animales,aventura,belico,biografia,ciencia,ciencia ficcion,cocina,comedia,competencia,crimen,cultura,deporte,dibujos animados,documental,drama,entretenimiento,entrevistas,espectaculo,familia,fantasia,historia,humor,infantil,interes general,investigacion,magazine,moda,musica,naturaleza,periodistico,policial,politico,reality,religion,restauracion,romance,suspenso,teatro,terror,viajes,western,title,keywords,ranking
